### Predicting transfer value of football players using ML (statistics and additional attributes)

In [51]:
# Importing relevant libaries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import YouTubeVideo
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup, NavigableString, Tag
import os


In [55]:
#Specify relevant seasons and leagues
years = ['2015-2016','2016-2017','2017-2018','2018-2019','2019-2020','2020-2021','2021-2022']
leagues = ['spanish-la-liga','italian-serie-a','german-bundesliga','english-premier-league','rest-of-europe']

#Test
years_test = ['2017-2018','2018-2019']
leagues_test = ['english-premier-league']

In [56]:
def scrape(driver,id):
    '''
    Sets up a function for scraping data based on a given driver

    args: driver (defines which driver to scrape from)
    '''

    soup = BeautifulSoup(driver.page_source, 'lxml')
    table_node = soup.find(id=id)

    columns_html = table_node.thead.find_all('th')
        # Extract the text
    columns = []
    for col in columns_html:
        columns.append(col.text)
    columns = columns[7:]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    for row_node in rows_list:
        # if (rows_list.index(row_node) % 25 == 0):
        try:
            row = []
            for child in row_node.children:
                row.append(child.text)
            data.append(row)
        except:
            pass
    
    return columns, data

def clean_statistics(columns,data,period):
    df_out = pd.DataFrame(data,columns=columns)\
        .iloc[:,:-1]\
        .assign(Nation= lambda x: x['Nation'].str.split(' ').str[1],
            Pos = lambda x: x['Pos'].str.split(',').str[0], 
            Comp = lambda x: x['Comp'].str.split(None,1).str[1],
            # Pos = lambda x: pd.Categorical(x['Pos']),
            Squad = lambda x: pd.Categorical(x['Squad']),
            Year = period[0:4])
            # Nation = lambda x: pd.Categorical(x['Nation']))

    return df_out

In [ ]:
def scrape(driver,id):
    '''
    Sets up a function for scraping data based on a given driver

    args: driver (defines which driver to scrape from)
    '''

    soup = BeautifulSoup(driver.page_source, 'lxml')
    table_node = soup.find(id=id)

    columns_html = table_node.thead.find_all('th')
        # Extract the text
    columns = []
    for col in columns_html:
        columns.append(col.text)
    columns = columns[7:]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    for row_node in rows_list:
        # if (rows_list.index(row_node) % 25 == 0):
        try:
            row = []
            for child in row_node.children:
                row.append(child.text)
            data.append(row)
        except:
            pass
    
    return columns, data

def clean_statistics(columns,data,period):
    df_out = pd.DataFrame(data,columns=columns)\
        .iloc[:,:-1]\
        .assign(Nation= lambda x: x['Nation'].str.split(' ').str[1],
            Pos = lambda x: x['Pos'].str.split(',').str[0], 
            Comp = lambda x: x['Comp'].str.split(None,1).str[1],
            # Pos = lambda x: pd.Categorical(x['Pos']),
            Squad = lambda x: pd.Categorical(x['Squad']),
            Year = period[0:4])
            # Nation = lambda x: pd.Categorical(x['Nation']))

    return df_out

In [58]:
df_list = []
id = 'stats_standard'
url = 'https://fbref.com/en/comps/Big5/2014-2015/stats/players/2014-2015-Big-5-European-Leagues-Stats'
driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
driver.get(url)
time.sleep(3)
cookie = driver.find_element(By.CLASS_NAME,'qc-cmp2-summary-buttons').click()


soup = BeautifulSoup(driver.page_source, 'lxml')
table_node = soup.find(id=id)

columns_html = table_node.thead.find_all('th')
    # Extract the text
columns = []
for col in columns_html:
    columns.append(col.text)
# columns = columns[7:]

# rows_list = table_node.tbody.find_all('tr')

# data = []
# for row_node in rows_list:
#     # if (rows_list.index(row_node) % 25 == 0):
#     try:
#         row = []
#         for child in row_node.children:
#             row.append(child.text)
#         data.append(row)
#     except:
#         pass




<ipython-input-58-cd743f028613>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver


In [41]:
df_stats['Year']= pd.to_datetime(df_stats['Year'], format='%Y')
df_stats['Year']= pd.DatetimeIndex(df_stats['Year']).year


In [33]:
def scrape_transferprice(driver,id):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    table_node = soup.find(id=id)

    columns_html = table_node.thead.find_all('th')
        # Extract the text
    transfer_columns = []
    for col in columns_html:
        transfer_columns.append(col.text)

    rows_list = table_node.tbody.find_all('tr')

    transfer_data = []
    for row_node in rows_list:
        row = []
        for child in row_node.children:
            try:
                row.append(child.text)
            except:
                pass
        transfer_data.append(row)
    
    return transfer_columns, transfer_data

def clean_transfer(columns,data,period):    
    transfer_df = pd.DataFrame(data, columns=columns)\
                .assign(When = lambda x: x['When'].str.split(' ').str[0],
                    Price = lambda x: x['Price'].str.split('(?:(.*\d))?(?:([a-zA-Z]+))?').str[1],
                    Player = lambda x: x['Player'].str.rsplit(' ').str[:-1].apply(' '.join),
                    Year = int(period[0:4])-1)\
                .query("When != 'Jan'")\
                .replace(to_replace='None', value=np.nan).dropna()

    transfer_df.drop(transfer_df.columns[[0,1,4]],axis=1, inplace=True)


    return transfer_df                         

In [34]:
def get_transferprice(leagues,years):
    id = 'epl'
    df_transferprice = []
    for league in leagues:

        df_transferprice_ = []
        
        for periods in years:
            url = f'https://www.soccernews.com/soccer-transfers/{league}-transfers-{periods}/'
            driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
            driver.get(url)
            # time.sleep(3)
            # cookie = driver.find_element(By.ID,'cn-accept-cookie').click()
            
            # Get data
            columns_transfer, transfer_data = scrape_transferprice(driver,id)

            df_transferprice_.append(clean_transfer(columns_transfer,transfer_data,periods))
    
        df_transferprice.append(pd.concat(df_transferprice_))

    return pd.concat(df_transferprice)
#     main_transfer = pd.concat(df_transferprice)
#     print(f'Number of observations in main dataset is: {len(main_transfer)}')

#     return main_transfer

# main_transfer_df = get_transferprice(leagues_test,years)

In [35]:
df_transferprice = get_transferprice(leagues,years)

<ipython-input-34-3900a32a9103>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
<ipython-input-34-3900a32a9103>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
<ipython-input-34-3900a32a9103>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
<ipython-input-34-3900a32a9103>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
<ipython-input-34-3900a32a9103>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #driver
<ipython-input-34-3900a32a9103

In [50]:
df_transferprice.to_csv('Transfer.csv')

In [43]:
df_transferprice['Year']= pd.to_datetime(df_transferprice['Year'], format='%Y')
df_transferprice['Year']= pd.DatetimeIndex(df_transferprice['Year']).year
df_transferprice.tail(10)

,Player,From,To,Price,Year
51,Marko Grujić,Liverpool,Porto,12,2020
53,Steven Berghuis,Feyenoord,Ajax,5.5,2020
55,Luan Peres,Santos,Marseille,4.5,2020
57,Calvin Stengs,AZ Alkmaar,Nice,15,2020
62,Ismail Jakobs,1. FC Koln,Monaco,6.5,2020
68,Achraf Hakimi,Inter Milan,PSG,60,2020
70,Loïc Badé,Lens,Rennes,20,2020
73,Jay Gorter,Go Ahead Eagles,Ajax,1,2020
76,Konrad de la Fuente,Barcelona,Marseille,3,2020
77,Danilo Pereira,FC Porto,PSG,16,2020


### Trying to merge transfer data with statistics

### **Hvis du har mod på det må du meget gerne kigge på dette, Carl Philip**

De to datasæt skal merges så vi ender med et samlet datasæt med stats og transferpris

In [36]:
def merge(stats,transfer):
    
    stats['Year']= pd.to_datetime(stats['Year'], format='%Y')
    stats['Year']= pd.DatetimeIndex(stats['Year']).year


    transfer['Year']= pd.to_datetime(transfer['Year'], format='%Y')
    transfer['Year']= pd.DatetimeIndex(transfer['Year']).year

    new_df = stats.merge(df_transferprice, how='left', on=['Player','Year'])
    
    new_df.dropna(subset=['Price'],inplace=True)

    return new_df

In [37]:
test = merge(df_stats,df_transferprice)

In [39]:
test.tail(10)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,npxG+xA,xG,xA,xG+xA,npxG,npxG+xA,Year,From,To,Price
5300,2479,Anthony Ujah,NGA,FW,Mainz 05,Bundesliga,27,1990,22,6,...,5.1,0.46,0.16,0.62,0.46,0.62,2018,Mainz,Union Berlin,1.80
5301,2479,Anthony Ujah,NGA,FW,Mainz 05,Bundesliga,27,1990,22,6,...,5.1,0.46,0.16,0.62,0.46,0.62,2018,Mainz,Union Berlin,1.80
5355,2533,Luciano Vietto,ARG,FW,Fulham,Premier League,24,1993,20,10,...,4.5,0.22,0.19,0.41,0.22,0.41,2018,Atletico Madrid,Sporting CP,6.75
5356,2533,Luciano Vietto,ARG,FW,Fulham,Premier League,24,1993,20,10,...,4.5,0.22,0.19,0.41,0.22,0.41,2018,Atletico Madrid,Sporting CP,6.75
5390,2567,Aaron Wan-Bissaka,ENG,DF,Crystal Palace,Premier League,20,1997,35,35,...,2.0,0.01,0.05,0.06,0.01,0.06,2018,Crystal Palace,Manchester United,53
5391,2567,Aaron Wan-Bissaka,ENG,DF,Crystal Palace,Premier League,20,1997,35,35,...,2.0,0.01,0.05,0.06,0.01,0.06,2018,Crystal Palace,Manchester United,53
5398,2574,Timothy Weah,USA,FW,Paris S-G,Ligue 1,18,2000,2,1,...,0.9,1.33,0.13,1.46,1.33,1.46,2018,PSG,Lille,9
5399,2574,Timothy Weah,USA,FW,Paris S-G,Ligue 1,18,2000,2,1,...,0.9,1.33,0.13,1.46,1.33,1.46,2018,PSG,Lille,9
5425,2600,Maximilian Wöber,AUT,DF,Sevilla,La Liga,20,1998,7,6,...,0.5,0.03,0.05,0.08,0.03,0.08,2018,Sevilla,RB Salzburg,12
5426,2600,Maximilian Wöber,AUT,DF,Sevilla,La Liga,20,1998,7,6,...,0.5,0.03,0.05,0.08,0.03,0.08,2018,Sevilla,RB Salzburg,12


In [19]:
test_stats_merge = stats_df.loc[stats_df['Player'] == 'Fabinho']

In [20]:
test_stats_merge

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,xG,npxG,xA,npxG+xA,xG,xA,xG+xA,npxG,npxG+xA,Year
778,779,Fabinho,BRA,MF,Monaco,Ligue 1,23,1993,34,34,...,5.4,2.4,3.8,6.2,0.16,0.11,0.27,0.07,0.18,2017
765,766,Fabinho,BRA,MF,Liverpool,Premier League,24,1993,28,21,...,0.8,0.8,1.3,2.1,0.04,0.06,0.09,0.04,0.09,2018


In [21]:
test_transfer_merge = df_transferprice.loc[df_transferprice['Player'] == 'Fabinho']

In [ ]:
#test_transfer_merge = test_transfer_merge.iloc[0]

In [46]:
#new_df = test_stats_merge.merge(test_transfer_merge, how='left', left_on=['Player','Year'], right_on=['Price'])

new_df = df_stats.merge(df_transferprice, how='left', on=['Player','Year'])
new_df.dropna(subset=['Price'],inplace=True)

# Nu skal der bare droppes de rows med NaN i price

In [48]:
new_df.tail(10)

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,npxG+xA,xG,xA,xG+xA,npxG,npxG+xA,Year,From,To,Price
5300,2479,Anthony Ujah,NGA,FW,Mainz 05,Bundesliga,27,1990,22,6,...,5.1,0.46,0.16,0.62,0.46,0.62,2018,Mainz,Union Berlin,1.80
5301,2479,Anthony Ujah,NGA,FW,Mainz 05,Bundesliga,27,1990,22,6,...,5.1,0.46,0.16,0.62,0.46,0.62,2018,Mainz,Union Berlin,1.80
5355,2533,Luciano Vietto,ARG,FW,Fulham,Premier League,24,1993,20,10,...,4.5,0.22,0.19,0.41,0.22,0.41,2018,Atletico Madrid,Sporting CP,6.75
5356,2533,Luciano Vietto,ARG,FW,Fulham,Premier League,24,1993,20,10,...,4.5,0.22,0.19,0.41,0.22,0.41,2018,Atletico Madrid,Sporting CP,6.75
5390,2567,Aaron Wan-Bissaka,ENG,DF,Crystal Palace,Premier League,20,1997,35,35,...,2.0,0.01,0.05,0.06,0.01,0.06,2018,Crystal Palace,Manchester United,53
5391,2567,Aaron Wan-Bissaka,ENG,DF,Crystal Palace,Premier League,20,1997,35,35,...,2.0,0.01,0.05,0.06,0.01,0.06,2018,Crystal Palace,Manchester United,53
5398,2574,Timothy Weah,USA,FW,Paris S-G,Ligue 1,18,2000,2,1,...,0.9,1.33,0.13,1.46,1.33,1.46,2018,PSG,Lille,9
5399,2574,Timothy Weah,USA,FW,Paris S-G,Ligue 1,18,2000,2,1,...,0.9,1.33,0.13,1.46,1.33,1.46,2018,PSG,Lille,9
5425,2600,Maximilian Wöber,AUT,DF,Sevilla,La Liga,20,1998,7,6,...,0.5,0.03,0.05,0.08,0.03,0.08,2018,Sevilla,RB Salzburg,12
5426,2600,Maximilian Wöber,AUT,DF,Sevilla,La Liga,20,1998,7,6,...,0.5,0.03,0.05,0.08,0.03,0.08,2018,Sevilla,RB Salzburg,12


### Celle til leg med plots (Carl Philip)